In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pymongo
import re
import time


In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
# Define database and collection
db = client.NavySchedule_db
collection = db.items

In [4]:
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36'}
res = requests.get('https://navysports.com/schedule.aspx?path=wlax', headers=headers)
soup = bs(res.content, 'html.parser')
soup.find_all('span', class_='sidearm-schedule-game-opponent-name')

[<span class="sidearm-schedule-game-opponent-name">
 <a aria-label="#16 Virginia on February 9 1:00 p.m." href="http://virginiasports.com/" target="_blank">#16 Virginia</a>
 </span>, <span class="sidearm-schedule-game-opponent-name">
 <a aria-label="Saint Joseph's on February 17 1:00 p.m." href="http://www.sjuhawks.com/" target="_blank">Saint Joseph's</a>
 </span>, <span class="sidearm-schedule-game-opponent-name">
 <a aria-label="#19 Towson on February 23 3:00 p.m." href="http://www.towsontigers.com/" target="_blank">#19 Towson</a>
 </span>, <span class="sidearm-schedule-game-opponent-name">
 <a aria-label="Villanova on February 27 3:00 p.m." href="http://www.villanova.com/" target="_blank">Villanova</a>
 </span>, <span class="sidearm-schedule-game-opponent-name">
 <a aria-label="#11 Florida on March 2 11:00 a.m." href="http://www.floridagators.com/" target="_blank">#11 Florida</a>
 </span>, <span class="sidearm-schedule-game-opponent-name">
 <a aria-label="#1 Boston College on March 

In [5]:
# Examine the results, then determine element that contains sought info
# results are returned as an iterable list
results = soup.find_all('span', class_='sidearm-schedule-game-opponent-name')

opponent = []

# Loop over td elements
for result in results:
            stuff = (result.a.text)
            clean_stuff= re.sub("\d", "",stuff)
            clean_stuff= re.sub("# ", "",clean_stuff)
            opponent.append(clean_stuff)

print(opponent)


['Virginia', "Saint Joseph's", 'Towson', 'Villanova', 'Florida', 'Boston College', 'Brown', 'Colgate', 'Bucknell', 'Lehigh', 'Holy Cross', 'Loyola', 'Georgetown', 'Army West Point', 'American', 'Lafayette', 'Boston University', 'Army West Point', 'Loyola', 'High Point', 'Virginia']


In [6]:
results2 = soup.find_all('span', class_='sidearm-schedule-game-conference-vs flex flex-inline')                                
  
home_away = []

# Loop over elements
for result2 in results2:
    if (result2.text):
            stuff = (result2.text)
            clean_stuff= re.sub("\n", "",stuff)
            home_away.append(clean_stuff)
            
print(home_away)

['at', 'vs', 'vs', 'at', 'vs', 'vs', 'vs', 'at', 'vs', 'at', 'vs', 'at', 'at', 'vs', 'vs', 'vs', 'at', 'vs', 'at', 'vs', 'at']


In [7]:
from datetime import datetime

results3 = soup.find_all('div', class_='sidearm-schedule-game-opponent-date flex-item-1')

period = []
day = []


# Loop over elements
for result3 in results3:
            stuff = ((result3.span.text)[0:3])
            clean_stuff = datetime.strptime(stuff,'%b')
            clean_stuff = clean_stuff.strftime('%m')
            period.append(clean_stuff)
            


            date_day = ((result3.span.text)[4:6])
            date_day = re.sub(" ", "",date_day)
            if len(date_day) == 1:
                date_day = "0" + date_day
            else:
                date_day = date_day
            day.append(date_day)   
            
print(period)          
print(day)   


['02', '02', '02', '02', '03', '03', '03', '03', '03', '03', '03', '04', '04', '04', '04', '04', '04', '05', '05', '05', '05']
['09', '17', '23', '27', '02', '06', '09', '16', '23', '27', '30', '06', '10', '13', '17', '20', '25', '02', '04', '10', '12']


In [8]:
import numpy as np
url_list = pd.DataFrame(np.column_stack([opponent,home_away,period,day]), columns=['key','Home_Away','Period','Day'])
url_list

,key,Home_Away,Period,Day
0,Virginia,at,02,09
1,Saint Joseph's,vs,02,17
2,Towson,vs,02,23
3,Villanova,at,02,27
4,Florida,vs,03,02
5,Boston College,vs,03,06
6,Brown,vs,03,09
7,Colgate,at,03,16
8,Bucknell,vs,03,23
9,Lehigh,at,03,27


In [9]:
# The path to our CSV file
file = "SystemSchool.csv"

# Read our Kickstarter data into pandas
df = pd.read_csv(file)
df.head()

,key,CONFERENCE,SystemName
0,Maryland,Big Ten,maryland
1,Boston College,ACC,boston-college
2,North Carolina,ACC,north-carolina
3,Northwestern,Big Ten,northwestern
4,Syracuse,ACC,syracuse


In [10]:
new_df = url_list.set_index('key').join(df.set_index('key'))

new_df = new_df.reset_index()

new_df = new_df.rename_axis("game_id")

new_df

,key,Home_Away,Period,Day,CONFERENCE,SystemName
game_id,,,,,,
0,American,vs,04,17,Patriot,american
1,Army West Point,vs,04,13,Patriot,army
2,Army West Point,vs,05,02,Patriot,army
3,Boston College,vs,03,06,ACC,boston-college
4,Boston University,at,04,25,Patriot,boston-u
5,Brown,vs,03,09,Ivy League,brown
6,Bucknell,vs,03,23,Patriot,bucknell
7,Colgate,at,03,16,Patriot,colgate
8,Florida,vs,03,02,AAC,florida


In [11]:
base = 'https://www.ncaa.com/game/lacrosse-women/d1/2019/'
p = '/play-by-play'
np = '-navy/play-by-play'

url_dict = []
    
for index, row in new_df.iterrows():
    if row['Home_Away'] == 'at':
        url = base + row['Period'] + '/' + row['Day'] + '/navy-' +  row['SystemName'] + p
        url_dict.append(url)
    else:
        url = base + row['Period'] + '/' + row['Day'] + '/' +  row['SystemName'] + np
        url_dict.append(url)

url_df = pd.DataFrame({"url_list": url_dict})
url_df




box_p = '/boxscore'
box_np = '-navy/boxscore'

box_url_dict = []
    
for index, row in new_df.iterrows():
    if row['Home_Away'] == 'at':
        url = base + row['Period'] + '/' + row['Day'] + '/navy-' +  row['SystemName'] + box_p
        box_url_dict.append(url)
    else:
        url = base + row['Period'] + '/' + row['Day'] + '/' +  row['SystemName'] + box_np
        box_url_dict.append(url)

box_url_df = pd.DataFrame({"url_list": box_url_dict})
box_url_df


,url_list
0,https://www.ncaa.com/game/lacrosse-women/d1/20...
1,https://www.ncaa.com/game/lacrosse-women/d1/20...
2,https://www.ncaa.com/game/lacrosse-women/d1/20...
3,https://www.ncaa.com/game/lacrosse-women/d1/20...
4,https://www.ncaa.com/game/lacrosse-women/d1/20...
5,https://www.ncaa.com/game/lacrosse-women/d1/20...
6,https://www.ncaa.com/game/lacrosse-women/d1/20...
7,https://www.ncaa.com/game/lacrosse-women/d1/20...
8,https://www.ncaa.com/game/lacrosse-women/d1/20...
9,https://www.ncaa.com/game/lacrosse-women/d1/20...


In [12]:
#import mysql.connector
from sqlalchemy import create_engine
from sqlalchemy.orm import Session

engine = create_engine('sqlite:///urldb', echo=False)
url_df.to_sql(name='sample_table2', con=engine, if_exists = 'replace', index=False)

In [13]:
session = Session(engine)
test = session.execute("select * from sample_table2")

result_proxy = test.fetchall()

#create list of urls
game_urls = []



for row in result_proxy:
    game_url = dict(row.items())
    print(game_url)
    game_urls.append(game_url["url_list"])
   





{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/04/17/american-navy/play-by-play'}
{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/04/13/army-navy/play-by-play'}
{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/05/02/army-navy/play-by-play'}
{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/03/06/boston-college-navy/play-by-play'}
{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/04/25/navy-boston-u/play-by-play'}
{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/03/09/brown-navy/play-by-play'}
{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/03/23/bucknell-navy/play-by-play'}
{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/03/16/navy-colgate/play-by-play'}
{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/03/02/florida-navy/play-by-play'}
{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/04/10/navy-georgetown/play-by-play'}
{'url_list': 'https://www

In [14]:
#import mysql.connector
from sqlalchemy import create_engine
from sqlalchemy.orm import Session

engine = create_engine('sqlite:///urldb', echo=False)
box_url_df.to_sql(name='sample_table3', con=engine, if_exists = 'replace', index=False)

In [15]:

session = Session(engine)
test = session.execute("select * from sample_table3")

result_proxy = test.fetchall()

#create list of urls
box_game_urls = []



for row in result_proxy:
    box_game_url = dict(row.items())
    print(box_game_url)
    box_game_urls.append(box_game_url["url_list"])


{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/04/17/american-navy/boxscore'}
{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/04/13/army-navy/boxscore'}
{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/05/02/army-navy/boxscore'}
{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/03/06/boston-college-navy/boxscore'}
{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/04/25/navy-boston-u/boxscore'}
{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/03/09/brown-navy/boxscore'}
{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/03/23/bucknell-navy/boxscore'}
{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/03/16/navy-colgate/boxscore'}
{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/03/02/florida-navy/boxscore'}
{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/04/10/navy-georgetown/boxscore'}
{'url_list': 'https://www.ncaa.com/game/lacrosse-women/d1/2019/05

In [16]:
## Now time for the web scraping of the box scores and play by play

In [17]:
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [18]:
print("Number of box score urls")
print(len(box_game_urls))
print("Number of play by play urls")
print(len(game_urls))

Number of box score urls
21
Number of play by play urls
21


In [19]:
game_id = 0

season_stats_df = pd.DataFrame(columns=['No', 'Name', 'POS', 'G', 'A', 'SH', 'SOG', 'GB','game_id', 'team'])
season_goalies_stats_df = pd.DataFrame(columns=['Goalies', 'POS', 'MIN', 'SAVES', 'GA', 'game_id', 'team'])
# Go to NCAA Boxscore through splinter module

for i in range(len(box_game_urls)):
        try:
            score_url = box_game_urls[i]
            browser.visit(score_url)

            html = browser.html
            soup = BeautifulSoup(html, 'lxml')
        
        
            away_tables = pd.read_html(html)
            away_team_selector = soup.find("div", {"class": "boxscore-team-selector-team awayTeam-bg-primary_color homeTeam-border-primary_color away active"})
            away_box_table = away_tables[1]
            away_goalies_box_table = away_tables[2]
            away_box_table['game_id'] = game_id
            away_box_table['team'] = away_team_selector.text
            away_goalies_box_table['game_id'] = game_id
            away_goalies_box_table['team'] = away_team_selector.text
        
        
            team_selectors = soup.find("div", {"class": "boxscore-team-selector"})
            xpath = '//div[@class="boxscore-team-selector-team homeTeam-bg-primary_color awayTeam-border-primary_color home"]'
            browser.find_by_xpath(xpath).click()
            html = browser.html
        
            home_tables = pd.read_html(html)
            home_team_selector = soup.find("div", {"class": "boxscore-team-selector-team homeTeam-bg-primary_color awayTeam-border-primary_color home"})
            home_box_table = home_tables[1]
            home_goalies_box_table = home_tables[2]
            home_box_table['game_id'] = game_id
            home_box_table['team'] = home_team_selector.text
            home_goalies_box_table['game_id'] = game_id
            home_goalies_box_table['team'] = home_team_selector.text

            season_stats_df = season_stats_df.append(away_box_table, sort=True)
            season_stats_df = season_stats_df.append(home_box_table, sort=True)
            season_goalies_stats_df = season_goalies_stats_df.append(away_goalies_box_table, sort=True)
            season_goalies_stats_df = season_goalies_stats_df.append(home_goalies_box_table, sort=True)
            game_id = game_id+1
         
        except:
            print(f"box score not retrievable for: {score_url}")
            pass



  

In [20]:
season_stats_df = season_stats_df.replace('\n','', regex=True)
season_goalies_stats_df = season_goalies_stats_df.replace('\n','', regex=True)

In [21]:
season_stats_df.head(100)

,A,G,GB,Name,No,POS,SH,SOG,game_id,team
0,1,1,0,CASEY HARKINS,3,M,3,2,0,American
1,0,3,0,KENDALL GOLDBLUM,18,M,4,4,0,American
2,0,1,3,AMANDA MEDJID,31,M,5,5,0,American
3,0,0,3,FIONA GEIER,17,GK,0,0,0,American
4,0,0,0,C AGUILAR-GERVASE,6,D,0,0,0,American
5,0,0,1,ASHLEY GO,10,D,0,0,0,American
6,0,0,1,CATE GOLDEN,22,D,0,0,0,American
7,0,0,2,SYDNEY FRY,23,D,0,0,0,American
8,0,0,3,KATIE WOOD,29,D,0,0,0,American
9,0,1,0,LEAH BRENNAN,1,A,4,3,0,American


In [22]:
season_goalies_stats_df.head(50)

,GA,Goalies,MIN,POS,SAVES,game_id,team
0,16,FIONA GEIER,NaN,GK,9,0,American
1,4,DELANEY OLIVEIRA,NaN,NaN,0,0,American
2,20,Total,NaN,Total,9,0,American
0,3,GAB HARCHELROAD,NaN,GK,3,0,Navy
1,1,LINDSEY ELLISON,NaN,NaN,0,0,Navy
2,2,ANNIE MALONEY,NaN,NaN,1,0,Navy
3,3,MADDY BELL,NaN,NaN,2,0,Navy
4,1,ABBI YOUNG,NaN,NaN,5,0,Navy
5,10,Total,NaN,Total,11,0,Navy
0,12,MADDIE BURNS,NaN,GK,5,1,Army


In [23]:
game_id = 0
season_play_by_play_df = pd.DataFrame(columns=['description', 'score', 'team', 'time','game_id'])

# Go to NCAA Play by play  through splinter module

for i in range(len(box_game_urls)):
    score_url = game_urls[i]
    browser.visit(score_url)
    html = browser.html
    #response = requests.get(url)
    soup = BeautifulSoup(html, 'lxml')
    all_rows = soup.find_all('tr')
    df_data = []
    for row in all_rows:
        try:
            img = row.find(class_="team")
            df_data.append({
                "time": row.find(class_="time").text,
                "team": img.find("img")["title"],
                "score": row.find(class_="score").text,
                "description": row.find(class_="description").text,
                "game_id": game_id
            })
            #print(row.find(class_="time").text)
            #print(img.find("img")["title"])
            #print(row.find(class_="score").text)
            #print(row.find(class_="description").text)
        except:
            #print(f"play by play not found {score_url}")
            pass
    game_id = game_id+1
    play_table_df = pd.DataFrame(df_data)
    season_play_by_play_df = season_play_by_play_df.append(play_table_df, sort=True)

In [24]:
season_play_by_play_df = season_play_by_play_df.replace('\n','', regex=True)

In [25]:
#join with schedule data for more complete dataset
season_play_by_play_df = season_play_by_play_df.join(new_df, on='game_id')

In [26]:
season_play_by_play_df.head(300)

,description,game_id,score,team,time,key,Home_Away,Period,Day,CONFERENCE,SystemName
0,"Geier, Fiona at goalie for AU.",0,,American,30:00,American,vs,04,17,Patriot,american
1,"Harchelroad, Gab at goalie for NAVY.",0,,Navy,30:00,American,vs,04,17,Patriot,american
2,"Draw control by AU Goldblum, Kendall.",0,,American,00:00,American,vs,04,17,Patriot,american
3,"Turnover by AU Jade, Izabella (caused by Heywa...",0,,American,29:26,American,vs,04,17,Patriot,american
4,"Ground ball pickup by NAVY Heyward, Annalise.",0,,Navy,29:23,American,vs,04,17,Patriot,american
5,Clear attempt by NAVY good.,0,,Navy,29:09,American,vs,04,17,Patriot,american
6,"GOAL by NAVY Victory, Nicole (FIRST GOAL), Ass...",0,0-1,Navy,28:56,American,vs,04,17,Patriot,american
7,"Draw control by NAVY Roelofs, Reagan.",0,,Navy,00:00,American,vs,04,17,Patriot,american
8,"GOAL by NAVY O'Sullivan, Andie, Assist by Heyw...",0,0-2,Navy,28:29,American,vs,04,17,Patriot,american
9,"Draw control by NAVY Harris, Kayla.",0,,Navy,00:00,American,vs,04,17,Patriot,american


In [27]:
#test sample recall of play by play for single game by game id
print(season_play_by_play_df.loc[season_play_by_play_df['game_id'] ==0])

                                           description  game_id  score  \
0                       Geier, Fiona at goalie for AU.        0          
1                 Harchelroad, Gab at goalie for NAVY.        0          
2                Draw control by AU Goldblum, Kendall.        0          
3    Turnover by AU Jade, Izabella (caused by Heywa...        0          
4        Ground ball pickup by NAVY Heyward, Annalise.        0          
5                          Clear attempt by NAVY good.        0          
6    GOAL by NAVY Victory, Nicole (FIRST GOAL), Ass...        0    0-1   
7                Draw control by NAVY Roelofs, Reagan.        0          
8    GOAL by NAVY O'Sullivan, Andie, Assist by Heyw...        0    0-2   
9                  Draw control by NAVY Harris, Kayla.        0          
10                           Foul on AU Brennan, Leah.        0          
11      Shot by NAVY Harris, Kayla, SAVE Geier, Fiona.        0          
12   GOAL by NAVY Larkin, Kelly, Assis

In [28]:
season_stats_df = season_stats_df.join(new_df, on='game_id')
season_goalies_stats_df = season_goalies_stats_df.join(new_df, on='game_id')

In [29]:
#test sample recall of box score for single game by game id

print(season_stats_df.loc[season_stats_df['game_id'] ==6])


    A  G  GB                Name  No  POS  SH  SOG  game_id      team  \
0   0  0   2       SOPHIA KLABER   9    M   0    0        6  Bucknell   
1   1  1   1  MORGAN STEINHACKER  12    M   3    2        6  Bucknell   
2   0  0   1     TACE SUTHERLAND  18    M   0    0        6  Bucknell   
3   0  0   2        JAMIE DEWITT  13   GK   0    0        6  Bucknell   
4   0  0   0       CALLIE DEWITT   7    D   0    0        6  Bucknell   
5   0  0   1         GABBY WALSH  15    D   0    0        6  Bucknell   
6   0  0   3      JESSIE BRITTON  30    D   0    0        6  Bucknell   
7   0  0   1    LYDIA MORGANTINI  32    D   0    0        6  Bucknell   
8   0  1   0    SARAH MONTELEONE   8    A   1    1        6  Bucknell   
9   0  0   0        AUDREY WALSH  21    A   3    1        6  Bucknell   
10  0  0   0         LILLIE HUNT  26    A   0    0        6  Bucknell   
11  0  0   0        KALI PEEPLES  27    A   2    0        6  Bucknell   
12  0  0   0        AVANI CHOPRA   1  NaN   0    0 

In [30]:
#test sample recall of goalies box score for single game by game id

print(season_goalies_stats_df.loc[season_goalies_stats_df['game_id'] ==7])

   GA           Goalies  MIN    POS  SAVES  game_id     team      key  \
0   5   GAB HARCHELROAD  NaN     GK      4        7     Navy  Colgate   
1   0   LINDSEY ELLISON  NaN    NaN      1        7     Navy  Colgate   
2   0        MADDY BELL  NaN    NaN      0        7     Navy  Colgate   
3   1        ABBI YOUNG  NaN    NaN      1        7     Navy  Colgate   
4   6             Total  NaN  Total      6        7     Navy  Colgate   
0  18  SAMANTHA CROSTON  NaN     GK     12        7  Colgate  Colgate   
1  18             Total  NaN  Total     12        7  Colgate  Colgate   

  Home_Away Period Day CONFERENCE SystemName  
0        at     03  16    Patriot    colgate  
1        at     03  16    Patriot    colgate  
2        at     03  16    Patriot    colgate  
3        at     03  16    Patriot    colgate  
4        at     03  16    Patriot    colgate  
0        at     03  16    Patriot    colgate  
1        at     03  16    Patriot    colgate  


In [31]:
#dump all of the data into mySQL

#import mysql.connector
from sqlalchemy import create_engine
from sqlalchemy.orm import Session

engine = create_engine('sqlite:///urldb', echo=False)


season_play_by_play_df.to_sql(name='season_play_by_play', con=engine, if_exists = 'replace', index=False)
season_stats_df.to_sql(name='season_stats', con=engine, if_exists = 'replace', index=False)
season_goalies_stats_df.to_sql(name='season_goalies_stats', con=engine, if_exists = 'replace', index=False)

In [32]:
#test the mySQL
session = Session(engine)
test = session.execute("select * from season_goalies_stats")

result_proxy = test.fetchall()

print(result_proxy)

[(16, 'FIONA GEIER', None, 'GK', 9, 0, 'American', 'American', 'vs', '04', '17', 'Patriot', 'american'), (4, 'DELANEY OLIVEIRA', None, None, 0, 0, 'American', 'American', 'vs', '04', '17', 'Patriot', 'american'), (20, 'Total', None, 'Total', 9, 0, 'American', 'American', 'vs', '04', '17', 'Patriot', 'american'), (3, 'GAB HARCHELROAD', None, 'GK', 3, 0, 'Navy', 'American', 'vs', '04', '17', 'Patriot', 'american'), (1, 'LINDSEY ELLISON', None, None, 0, 0, 'Navy', 'American', 'vs', '04', '17', 'Patriot', 'american'), (2, 'ANNIE MALONEY', None, None, 1, 0, 'Navy', 'American', 'vs', '04', '17', 'Patriot', 'american'), (3, 'MADDY BELL', None, None, 2, 0, 'Navy', 'American', 'vs', '04', '17', 'Patriot', 'american'), (1, 'ABBI YOUNG', None, None, 5, 0, 'Navy', 'American', 'vs', '04', '17', 'Patriot', 'american'), (10, 'Total', None, 'Total', 11, 0, 'Navy', 'American', 'vs', '04', '17', 'Patriot', 'american'), (12, 'MADDIE BURNS', None, 'GK', 5, 1, 'Army', 'Army West Point', 'vs', '04', '13', '

In [33]:
class Game():
    def __init__(self):
        self.game_id = 0
        self.shots = 0
        self.draw_controls = 0
        self.fouls = 0
        self.free_position_attempts = 0
        self.turnovers = 0
        self.ground_ball_pickups = 0
        self.clear_attempts = 0
        self.goals = 0
        self.timeouts = 0
        self.Navy_shots = 0
        self.opponent_shots = 0
        self.Navy_draw_controls = 0
        self.opponent_draw_controls = 0
        self.Navy_fouls = 0
        self.opponent_fouls = 0
        self.Navy_free_positions = 0
        self.opponent_free_positions = 0
        self.Navy_turnovers = 0
        self.opponent_turnovers = 0
        self.Navy_ground_ball_pickups = 0
        self.opponent_ground_ball_pickups = 0
        self.Navy_clear_attempts = 0
        self.Navy_successful_clears = 0
        self.Navy_failed_clears = 0
        self.opponent_clear_attempts = 0
        self.opponent_successful_clears = 0
        self.opponent_failed_clears = 0
        self.Navy_goals = 0
        self.opponent_goals = 0
        self.Navy_timeouts = 0
        self.opponent_timeouts = 0
        self.possessions = 0
        self.Navy_possessions = 0
        self.opponent_possessions = 0
    
    def print(self):
        attrs = vars(self)
        print(', '.join("%s: %s" % item for item in attrs.items()))

    def newPos(self):
        print("Adding new pos")
    
    def as_dict(self):
        return {'game_id': self.game_id,
                'shots': self.shots, 
                'draw_controls': self.draw_controls, 
                'fouls': self.fouls, 
                'free_position_attempts' : self.free_position_attempts,
                'turnovers' : self.turnovers,
                'ground_ball_pickups' : self.ground_ball_pickups,
                'clear_attempts' : self.clear_attempts,
                'goals' : self.goals,
                'timeouts' : self.timeouts,
                'Navy_shots' : self.Navy_shots,
                'opponent_shots' : self.opponent_shots,
                'Navy_draw_controls' : self.Navy_draw_controls,
                'opponent_draw_controls' : self.opponent_draw_controls,
                'Navy_fouls' : self.Navy_fouls,
                'opponent_fouls' : self.opponent_fouls,
                'Navy_free_positions' : self.Navy_free_positions,
                'opponent_free_positions' : self.opponent_free_positions,
                'Navy_turnovers' : self.Navy_turnovers,
                'opponent_turnovers' : self.opponent_turnovers,
                'Navy_ground_ball_pickups' : self.Navy_ground_ball_pickups,
                'opponent_ground_ball_pickups' : self.opponent_ground_ball_pickups,
                'Navy_clear_attempts' : self.Navy_clear_attempts,
                'Navy_successful_clears' : self.Navy_successful_clears,
                'Navy_failed_clears' : self.Navy_failed_clears,
                'opponent_clear_attempts' : self.opponent_clear_attempts,
                'opponent_successful_clears' : self.opponent_successful_clears,
                'opponent_failed_clears' : self.opponent_failed_clears,
                'Navy_goals' : self.Navy_goals,
                'opponent_goals' : self.opponent_goals,
                'Navy_timeouts' : self.Navy_timeouts,
                'opponent_timeouts' : self.opponent_timeouts,
                'possessions' : self.possessions,
                'Navy_possessions' : self.Navy_possessions,
                'opponent_possessions' : self.opponent_possessions}
        
class Possession():
    
    def __init__(self):
        game_id = 0
        team = "n/a"
        cause = "n/a"
        result = "n/a"
        start_time = 0
        end_time = 0
        
    def print(self):
        attrs = vars(self)
        print(', '.join("%s: %s" % item for item in attrs.items()))
        
    def as_dict(self):
        return {'game_id': self.game_id,
                'team': self.team, 
                'cause': self.cause, 
                'result': self.result, 
                'start_time' : self.start_time,
                'end_time' : self.end_time,
               }


In [34]:
play_by_play_df = season_play_by_play_df.loc[season_play_by_play_df['game_id'] == i]

play_by_play_df.head()

,description,game_id,score,team,time,key,Home_Away,Period,Day,CONFERENCE,SystemName
0,"Harchelroad, Gab at goalie for NAVY.",20,,Navy,30:00,Virginia,at,05,12,ACC,virginia
1,Charlie Campbell at goalie for VA.,20,,Virginia,30:00,Virginia,at,05,12,ACC,virginia
2,"Draw control by NAVY Heyward, Annalise.",20,,Navy,00:00,Virginia,at,05,12,ACC,virginia
3,"Turnover by NAVY Larkin, Kelly (caused by Alli...",20,,Navy,29:16,Virginia,at,05,12,ACC,virginia
4,Ground ball pickup by VA Ashley Stilo.,20,,Virginia,29:12,Virginia,at,05,12,ACC,virginia


In [35]:
#use this if you want to do a dictionary
#games = {}

#use this if you want to do a list
games = []

possessions = []

for i in range(len(box_game_urls)):
    
    play_by_play_df = season_play_by_play_df.loc[season_play_by_play_df['game_id'] == i]

    game = Game()
    possession = Possession()
    

    
    
    for j in range(len(play_by_play_df)):
        current_row = play_by_play_df.iloc[j]
        current_description = current_row['description']
        current_time = current_row['time']
        print(current_time)
        print(current_description)
        possession.print()
        list_of_words = current_description.split()
        game.game_id = i
        possession.game_id = game.game_id
        
        if (current_description.find('Shot') != -1): 
                search_word = "by"
                game.shots = game.shots + 1
                possession.result = "Shot"
                possession.end_time = current_time
                
                
                
                shooting_team = list_of_words[list_of_words.index(search_word) + 1]

                if shooting_team == "NAVY":
                    
                    game.Navy_shots = game.Navy_shots + 1
                    
                    possession.team = "Navy"
                    possessions.append(possession)
                    possession.print()
                    possession = Possession ()
                    possession.game_id = game.game_id
                    possession.cause = "Shot"
                    possession.start_time = current_time
                    possession.team = "Navy"
                    possession.print()
                    
                else:

                    game.opponent_shots = game.opponent_shots + 1
                    possession.team = "opponent"
                    possessions.append(possession)
                    possession.print()
                    possession = Possession ()
                    possession.game_id = game.game_id
                    possession.cause = "Shot"
                    possession.start_time = current_time
                    possession.team = "opponent"
                    possession.print()
                    
        else: 
                game.shots = game.shots



        if (current_description.find('Draw control') != -1): 
                search_word = "by"
                game.draw_controls = game.draw_controls + 1
                game.possessions = game.possessions + 1
                possession.cause = "Draw"
                possession.start_time = current_time
                
                
                
                draw_control_team = list_of_words[list_of_words.index(search_word) + 1]

                if draw_control_team == "NAVY":

                    game.Navy_draw_controls = game.Navy_draw_controls + 1
                    game.Navy_possessions = game.Navy_possessions + 1
                    possession.team = "Navy"
                    possession.print()
                    
                else:

                    game.opponent_draw_controls = game.opponent_draw_controls + 1
                    game.opponent_possessions = game.opponent_possessions + 1
                    possession.team = "opponent"
                    possession.print()
        else: 
                game.draw_controls = game.draw_controls




        if (current_description.find('Foul') != -1): 
                search_word = "on"
                game.fouls = game.fouls + 1
                foul_on_team = list_of_words[list_of_words.index(search_word) + 1]

                if foul_on_team == "NAVY":

                    game.Navy_fouls = game.Navy_fouls + 1

                else:

                    game.opponent_fouls = game.opponent_fouls + 1

        else: 
                game.fouls = game.fouls





        if (current_description.find('Free position attempt') != -1): 
                search_word = "for"
                game.free_position_attempts = game.free_position_attempts + 1
                free_position_team = list_of_words[list_of_words.index(search_word) + 1]
                #print(free_position_team)
                if free_position_team == "NAVY.":

                    game.Navy_free_positions = game.Navy_free_positions + 1

                else:

                    game.opponent_free_positions = game.opponent_free_positions + 1

        else: 
                game.free_position_attempts = game.free_position_attempts



        if (current_description.find('Turnover') != -1): 
                search_word = "by"
                game.turnovers = game.turnovers + 1
                turnover_team = list_of_words[list_of_words.index(search_word) + 1]
                possession.result = "Turnover"
                possession.end_time = current_time

                if turnover_team == "NAVY":

                    game.Navy_turnovers = game.Navy_turnovers + 1
                    game.opponent_possessions = game.opponent_possessions + 1
                    game.possessions = game.possessions + 1
                    
                    possession.team = "Navy"
                    possessions.append(possession)
                    possession.print()
                    possession = Possession ()
                    possession.team = "opponent"
                    possession.game_id = game.game_id
                    possession.cause = "Turnover"
                    possession.start_time = current_time
                    possession.print()
                    
                else:

                    game.opponent_turnovers = game.opponent_turnovers + 1
                    game.Navy_possessions = game.Navy_possessions + 1
                    game.possessions = game.possessions + 1
                    possession.team = "opponent"
                    possessions.append(possession)
                    possession.print()
                    possession = Possession ()
                    possession.team = "Navy"
                    possession.game_id = game.game_id
                    possession.cause = "Turnover"
                    possession.start_time = current_time
                    possession.print()
                    
        else: 
                game.turnovers = game.turnovers



        if (current_description.find('Ground ball pickup') != -1): 
                search_word = "by"
                game.ground_ball_pickups = game.ground_ball_pickups + 1
                ground_ball_pickup_team = list_of_words[list_of_words.index(search_word) + 1]
                
                possession.cause = "Turnover"
                possession.start_time = current_time
                
                
                if ground_ball_pickup_team == "NAVY":

                    game.Navy_ground_ball_pickups = game.Navy_ground_ball_pickups + 1
                    possession.team = "Navy"
                    possession.print()
                    
                else:

                    game.opponent_ground_ball_pickups = game.opponent_ground_ball_pickups + 1   
                    possession.team = "opponent"
                    possession.print()
                    
        else: 
                game.ground_ball_pickups = game.ground_ball_pickups


        if (current_description.find('Clear attempt') != -1): 
                search_word = "by"
                game.clear_attempts = game.clear_attempts + 1
                clear_attempt_team = list_of_words[list_of_words.index(search_word) + 1]
                pass_or_fail_status = list_of_words[-1]
                #print(pass_or_fail_status)
                possession.result = "Clear"
                possession.end_time = current_time
                possessions.append(possession)
                possession.print()
                possession = Possession ()
                possession.game_id = game.game_id
                possession.cause = "Clear"
                possession.start_time = current_time
                
                if clear_attempt_team == "NAVY":

                    game.Navy_clear_attempts = game.Navy_clear_attempts + 1

                    if pass_or_fail_status == "good.":
                        possession.team = "Navy"
                        game.Navy_successful_clears = game.Navy_successful_clears + 1
                        game.Navy_possessions = game.Navy_possessions + 1
                        possession.print()
                        
                    else:
                        possession.team = "opponent"
                        game.Navy_failed_clears = game.Navy_failed_clears +1
                        game.opponent_possessions = game.opponent_possessions + 1
                        possession.print()
                else:

                    game.opponent_clear_attempts = game.opponent_clear_attempts + 1   

                    if pass_or_fail_status == "good.":

                        possession.team = "opponent"
                        game.opponent_successful_clears = game.opponent_successful_clears + 1
                        game.opponent_possessions = game.opponent_possessions + 1
                        possession.print()
                        
                    else:
                        possession.team = "Navy"
                        game.opponent_failed_clears = game.opponent_failed_clears + 1
                        game.Navy_possessions = game.Navy_possessions + 1
                        possession.print()

        else: 
                game.clear_attempts = game.clear_attempts


        if (current_description.find('GOAL') != -1): 
                search_word = "by"
                game.goals = game.goals + 1
                scoring_team = list_of_words[list_of_words.index(search_word) + 1]
                possession.result = "Goal"
                possession.end_time = current_time
                
                
                
                if scoring_team == "NAVY":

                    game.Navy_goals = game.Navy_goals + 1
                    possession.team = "Navy"
                    possessions.append(possession)
                    possession.print()
                    possession = Possession ()
                    possession.cause = "goal"
                    possession.game_id = game.game_id
                    possession.start_time = current_time
                    possession.print()
                    
                else:

                    game.opponent_goals = game.opponent_goals + 1   
                    possession.team = "opponent"
                    
                    possessions.append(possession)
                    possession.print()
                    possession = Possession ()
                    possession.cause = "goal"
                    possession.game_id = game.game_id
                    possession.start_time = current_time
                    possession.print()

        else: 
                game.goals = game.goals



        if (current_description.find('Timeout') != -1): 
                search_word = "by"
                game.timeouts = game.timeouts + 1
                timeout_team = list_of_words[list_of_words.index(search_word) + 1]
                #print(timeout_team)
                if timeout_team == "NAVY.":

                    game.Navy_timeouts = game.Navy_timeouts + 1

                else:

                    game.opponent_timeouts = game.opponent_timeouts + 1   


        else: 
                game.timeouts = game.timeouts
    
    #use this if you want to do this as a dictionary
    #games['game_{}'.format(i)] = game
    
    #use this if you want to do this as a list
    games.append(game)
            

30:00
Geier, Fiona at goalie for AU.

30:00
Harchelroad, Gab at goalie for NAVY.
game_id: 0
00:00
Draw control by AU Goldblum, Kendall.
game_id: 0
game_id: 0, cause: Draw, start_time: 00:00, team: opponent
29:26
Turnover by AU Jade, Izabella (caused by Heyward, Annalise).
game_id: 0, cause: Draw, start_time: 00:00, team: opponent
game_id: 0, cause: Draw, start_time: 00:00, team: opponent, result: Turnover, end_time: 29:26
team: Navy, game_id: 0, cause: Turnover, start_time: 29:26
29:23
Ground ball pickup by NAVY Heyward, Annalise.
team: Navy, game_id: 0, cause: Turnover, start_time: 29:26
team: Navy, game_id: 0, cause: Turnover, start_time: 29:23
29:09
Clear attempt by NAVY good.
team: Navy, game_id: 0, cause: Turnover, start_time: 29:23
team: Navy, game_id: 0, cause: Turnover, start_time: 29:23, result: Clear, end_time: 29:09
game_id: 0, cause: Clear, start_time: 29:09, team: Navy
28:56
GOAL by NAVY Victory, Nicole (FIRST GOAL), Assist by Roelofs, Reagan, goal number 15 for season.
ga

30:00
Burns, Maddie at goalie for ARMY.

30:00
Harchelroad, Gab at goalie for NAVY.
game_id: 1
00:00
Draw control by NAVY Roelofs, Reagan.
game_id: 1
game_id: 1, cause: Draw, start_time: 00:00, team: Navy
00:00
Foul on ARMY Crump, Nia.
game_id: 1, cause: Draw, start_time: 00:00, team: Navy
29:17
Free position attempt for NAVY.
game_id: 1, cause: Draw, start_time: 00:00, team: Navy
28:57
Turnover by NAVY Larkin, Kelly.
game_id: 1, cause: Draw, start_time: 00:00, team: Navy
game_id: 1, cause: Draw, start_time: 00:00, team: Navy, result: Turnover, end_time: 28:57
team: opponent, game_id: 1, cause: Turnover, start_time: 28:57
28:54
Ground ball pickup by ARMY Crump, Nia.
team: opponent, game_id: 1, cause: Turnover, start_time: 28:57
team: opponent, game_id: 1, cause: Turnover, start_time: 28:54
28:48
Clear attempt by ARMY failed.
team: opponent, game_id: 1, cause: Turnover, start_time: 28:54
team: opponent, game_id: 1, cause: Turnover, start_time: 28:54, result: Clear, end_time: 28:48
game_

Turnover by ARMY Scott, Rilee (caused by Irwin, Natalie).
game_id: 1, cause: Shot, start_time: 09:04, team: opponent
game_id: 1, cause: Shot, start_time: 09:04, team: opponent, result: Turnover, end_time: 08:52
team: Navy, game_id: 1, cause: Turnover, start_time: 08:52
08:49
Ground ball pickup by NAVY Harchelroad, Gab.
team: Navy, game_id: 1, cause: Turnover, start_time: 08:52
team: Navy, game_id: 1, cause: Turnover, start_time: 08:49
08:39
Clear attempt by NAVY good.
team: Navy, game_id: 1, cause: Turnover, start_time: 08:49
team: Navy, game_id: 1, cause: Turnover, start_time: 08:49, result: Clear, end_time: 08:39
game_id: 1, cause: Clear, start_time: 08:39, team: Navy
07:47
GOAL by NAVY Harris, Kayla, Assist by Victory, Nicole, goal number 27 for season.
game_id: 1, cause: Clear, start_time: 08:39, team: Navy
game_id: 1, cause: Clear, start_time: 08:39, team: Navy, result: Goal, end_time: 07:47
cause: goal, game_id: 1, start_time: 07:47
07:47
Media timeout.
cause: goal, game_id: 1, s

Yellow card on ARMY Andrews, Taylor.
game_id: 2, cause: Turnover, start_time: 26:11, team: Navy
25:57
Clear attempt by NAVY good.
game_id: 2, cause: Turnover, start_time: 26:11, team: Navy
game_id: 2, cause: Turnover, start_time: 26:11, team: Navy, result: Clear, end_time: 25:57
game_id: 2, cause: Clear, start_time: 25:57, team: Navy
25:26
Shot by NAVY Roelofs, Reagan WIDE.
game_id: 2, cause: Clear, start_time: 25:57, team: Navy
game_id: 2, cause: Clear, start_time: 25:57, team: Navy, result: Shot, end_time: 25:26
game_id: 2, cause: Shot, start_time: 25:26, team: Navy
25:24
Ground ball pickup by NAVY Victory, Nicole.
game_id: 2, cause: Shot, start_time: 25:26, team: Navy
game_id: 2, cause: Turnover, start_time: 25:24, team: Navy
00:00
Foul on ARMY Bredenburg, Lauren.
game_id: 2, cause: Turnover, start_time: 25:24, team: Navy
25:15
Shot by NAVY Harris, Kayla, SAVE Burns, Maddie.
game_id: 2, cause: Turnover, start_time: 25:24, team: Navy
game_id: 2, cause: Turnover, start_time: 25:24, te

02:45
Turnover by NAVY Victory, Nicole.
cause: Draw, game_id: 2, start_time: 00:00, team: Navy
cause: Draw, game_id: 2, start_time: 00:00, team: Navy, result: Turnover, end_time: 02:45
team: opponent, game_id: 2, cause: Turnover, start_time: 02:45
02:43
Ground ball pickup by ARMY Stewart, Samantha.
team: opponent, game_id: 2, cause: Turnover, start_time: 02:45
team: opponent, game_id: 2, cause: Turnover, start_time: 02:43
02:31
Clear attempt by ARMY good.
team: opponent, game_id: 2, cause: Turnover, start_time: 02:43
team: opponent, game_id: 2, cause: Turnover, start_time: 02:43, result: Clear, end_time: 02:31
game_id: 2, cause: Clear, start_time: 02:31, team: opponent
02:06
Shot by ARMY Brattan, Jackie, SAVE Harchelroad, Gab.
game_id: 2, cause: Clear, start_time: 02:31, team: opponent
game_id: 2, cause: Clear, start_time: 02:31, team: opponent, result: Shot, end_time: 02:06
game_id: 2, cause: Shot, start_time: 02:06, team: opponent
02:04
Ground ball pickup by NAVY Harchelroad, Gab.
ga

...

GOAL by LOY Taylor VanThof, Assist by Livy Rosenzweig, goal number 16 for season.
game_id: 14, cause: Clear, start_time: 07:58, team: opponent
game_id: 14, cause: Clear, start_time: 07:58, team: opponent, result: Goal, end_time: 07:42
cause: goal, game_id: 14, start_time: 07:42
07:42
Timeout by NAVY.
cause: goal, game_id: 14, start_time: 07:42
00:00
Draw control by NAVY Harris, Kayla.
cause: goal, game_id: 14, start_time: 07:42
cause: Draw, game_id: 14, start_time: 00:00, team: Navy
00:00
Foul on LOY Katie Korvin.
cause: Draw, game_id: 14, start_time: 00:00, team: Navy
06:36
Shot by NAVY Lazarides, Marina WIDE RIGHT.
cause: Draw, game_id: 14, start_time: 00:00, team: Navy
cause: Draw, game_id: 14, start_time: 00:00, team: Navy, result: Shot, end_time: 06:36
game_id: 14, cause: Shot, start_time: 06:36, team: Navy
06:28
Shot by NAVY Victory, Nicole, SAVE Kady Glynn.
game_id: 14, cause: Shot, start_time: 06:36, team: Navy
game_id: 14, cause: Shot, start_time: 06:36, team: Navy, result: Sh

team: opponent, game_id: 15, cause: Turnover, start_time: 04:09
team: opponent, game_id: 15, cause: Turnover, start_time: 04:09, result: Clear, end_time: 03:45
game_id: 15, cause: Clear, start_time: 03:45, team: opponent
00:00
Foul on NAVY Kwon, Caroline.
game_id: 15, cause: Clear, start_time: 03:45, team: opponent
02:51
Free position attempt for LOY.
game_id: 15, cause: Clear, start_time: 03:45, team: opponent
00:00
Foul on NAVY Downey, Hannah.
game_id: 15, cause: Clear, start_time: 03:45, team: opponent
02:25
Free position attempt for LOY.
game_id: 15, cause: Clear, start_time: 03:45, team: opponent
02:23
GOAL by LOY Livy Rosenzweig, Assist by Sam Fiedler, goal number 42 for season.
game_id: 15, cause: Clear, start_time: 03:45, team: opponent
game_id: 15, cause: Clear, start_time: 03:45, team: opponent, result: Goal, end_time: 02:23
cause: goal, game_id: 15, start_time: 02:23
00:00
Draw control by LOY Sam Fiedler.
cause: goal, game_id: 15, start_time: 02:23
cause: Draw, game_id: 15, 

Foul on TUWLAX19 Chenoweth, Sami.
game_id: 17, cause: Clear, start_time: 15:40, team: Navy
14:53
Free position attempt for NAVY.
game_id: 17, cause: Clear, start_time: 15:40, team: Navy
14:53
Shot by NAVY Harris, Kayla BLOCKED {free position shot}.
game_id: 17, cause: Clear, start_time: 15:40, team: Navy
game_id: 17, cause: Clear, start_time: 15:40, team: Navy, result: Shot, end_time: 14:53
game_id: 17, cause: Shot, start_time: 14:53, team: Navy
14:49
Ground ball pickup by NAVY Larkin, Kelly.
game_id: 17, cause: Shot, start_time: 14:53, team: Navy
game_id: 17, cause: Turnover, start_time: 14:49, team: Navy
00:00
Foul on TUWLAX19 Conti, Olivia.
game_id: 17, cause: Turnover, start_time: 14:49, team: Navy
14:44
Free position attempt for NAVY.
game_id: 17, cause: Turnover, start_time: 14:49, team: Navy
14:32
GOAL by NAVY Harris, Kayla, Assist by Larkin, Kelly, goal number 9 for season.
game_id: 17, cause: Turnover, start_time: 14:49, team: Navy
game_id: 17, cause: Turnover, start_time: 14:

cause: Draw, game_id: 18, start_time: 00:00, team: opponent
cause: Draw, game_id: 18, start_time: 00:00, team: opponent, result: Shot, end_time: 29:55
game_id: 18, cause: Shot, start_time: 29:55, team: opponent
29:40
Clear attempt by NAVY good.
game_id: 18, cause: Shot, start_time: 29:55, team: opponent
game_id: 18, cause: Shot, start_time: 29:55, team: opponent, result: Clear, end_time: 29:40
game_id: 18, cause: Clear, start_time: 29:40, team: Navy
29:22
Shot by NAVY Larkin, Kelly, SAVE MORO, Alexa.
game_id: 18, cause: Clear, start_time: 29:40, team: Navy
game_id: 18, cause: Clear, start_time: 29:40, team: Navy, result: Shot, end_time: 29:22
game_id: 18, cause: Shot, start_time: 29:22, team: Navy
29:08
Clear attempt by VU failed.
game_id: 18, cause: Shot, start_time: 29:22, team: Navy
game_id: 18, cause: Shot, start_time: 29:22, team: Navy, result: Clear, end_time: 29:08
game_id: 18, cause: Clear, start_time: 29:08, team: Navy
29:07
Turnover by VU MENTZER, Kelsey (caused by Larkin, Ke

game_id: 19, cause: Shot, start_time: 06:21, team: Navy
game_id: 19, cause: Shot, start_time: 06:21, team: Navy, result: Goal, end_time: 04:57
cause: goal, game_id: 19, start_time: 04:57
04:57
Timeout by VA.
cause: goal, game_id: 19, start_time: 04:57
00:00
Draw control by NAVY Harris, Kayla.
cause: goal, game_id: 19, start_time: 04:57
cause: Draw, game_id: 19, start_time: 00:00, team: Navy
00:00
Foul on VA Courtlynne Caskin.
cause: Draw, game_id: 19, start_time: 00:00, team: Navy
00:00
Foul on VA Courtlynne Caskin.
cause: Draw, game_id: 19, start_time: 00:00, team: Navy
04:01
Shot by NAVY Roelofs, Reagan, SAVE Charlie Campbell {free position shot}.
cause: Draw, game_id: 19, start_time: 00:00, team: Navy
cause: Draw, game_id: 19, start_time: 00:00, team: Navy, result: Shot, end_time: 04:01
game_id: 19, cause: Shot, start_time: 04:01, team: Navy
03:26
Clear attempt by VA good.
game_id: 19, cause: Shot, start_time: 04:01, team: Navy
game_id: 19, cause: Shot, start_time: 04:01, team: Navy

In [36]:
current_game = games[14]

current_game.print()

game_id: 14, shots: 27, draw_controls: 24, fouls: 38, free_position_attempts: 15, turnovers: 28, ground_ball_pickups: 34, clear_attempts: 39, goals: 22, timeouts: 4, Navy_shots: 18, opponent_shots: 9, Navy_draw_controls: 16, opponent_draw_controls: 8, Navy_fouls: 19, opponent_fouls: 19, Navy_free_positions: 4, opponent_free_positions: 11, Navy_turnovers: 15, opponent_turnovers: 13, Navy_ground_ball_pickups: 16, opponent_ground_ball_pickups: 18, Navy_clear_attempts: 16, Navy_successful_clears: 12, Navy_failed_clears: 4, opponent_clear_attempts: 23, opponent_successful_clears: 21, opponent_failed_clears: 2, Navy_goals: 6, opponent_goals: 16, Navy_timeouts: 2, opponent_timeouts: 2, possessions: 52, Navy_possessions: 43, opponent_possessions: 48


In [37]:
current_possession = possessions[0]
print(current_possession.print())
print(current_possession.cause)
print(possessions[541].cause)

game_id: 0, cause: Draw, start_time: 00:00, team: opponent, result: Turnover, end_time: 29:26
None
Draw
Shot


In [38]:
print(current_game.game_id)

14


In [39]:
games[12].opponent_successful_clears

15

In [40]:
df = pd.DataFrame([x.as_dict() for x in games])

In [41]:
df1 =  df[['game_id','shots']]
df1.head()

,game_id,shots
0,0,35
1,1,26
2,2,43
3,3,33
4,4,33


In [42]:
df.head(21)

,Navy_clear_attempts,Navy_draw_controls,Navy_failed_clears,Navy_fouls,Navy_free_positions,Navy_goals,Navy_ground_ball_pickups,Navy_possessions,Navy_shots,Navy_successful_clears,...,opponent_ground_ball_pickups,opponent_possessions,opponent_shots,opponent_successful_clears,opponent_timeouts,opponent_turnovers,possessions,shots,timeouts,turnovers
0,19,19,2,10,7,20,19,56,16,17,...,17,41,19,13,1,15,60,35,1,29
1,19,17,1,16,5,21,21,58,17,18,...,10,39,9,12,1,20,63,26,1,32
2,13,26,0,14,10,20,15,48,22,13,...,9,29,21,16,0,8,47,43,0,15
3,21,7,3,12,1,5,17,43,17,18,...,16,50,16,17,0,15,52,33,2,27
4,18,16,2,15,10,13,18,52,17,16,...,18,49,16,19,3,15,59,33,5,32
5,24,21,2,24,12,18,15,56,21,22,...,16,49,24,22,2,10,56,45,2,24
6,21,19,2,12,11,17,17,67,30,19,...,14,35,14,20,2,20,52,44,2,32
7,23,11,5,0,12,18,28,52,18,18,...,14,47,15,14,1,21,60,33,1,34
8,23,15,5,9,7,14,18,49,15,18,...,19,57,19,20,3,15,62,34,6,34
9,22,16,3,8,10,16,19,51,16,19,...,18,46,16,16,2,15,58,32,2,31


In [43]:
file_name = 'game_summary_stats.csv'
df.to_csv(file_name, encoding='utf-8')

In [44]:
df2 = pd.DataFrame([x.as_dict() for x in possessions])
df2.describe()
df2.head(625)

,cause,end_time,game_id,result,start_time,team
0,Draw,29:26,0,Turnover,00:00,opponent
1,Turnover,29:09,0,Clear,29:23,Navy
2,Clear,28:56,0,Goal,29:09,Navy
3,Draw,28:29,0,Goal,00:00,Navy
4,Draw,27:33,0,Shot,00:00,Navy
5,Shot,27:25,0,Goal,27:33,Navy
6,Draw,27:00,0,Goal,00:00,Navy
7,Draw,26:20,0,Shot,00:00,Navy
8,Shot,26:18,0,Goal,26:20,Navy
9,Draw,25:27,0,Goal,00:00,Navy


In [45]:
file_name = 'possession_data.csv'
df2.to_csv(file_name, encoding='utf-8')

In [46]:
pd.options.display.max_colwidth = 100
print(box_url_df['url_list'])

0            https://www.ncaa.com/game/lacrosse-women/d1/2019/04/17/american-navy/boxscore
1                https://www.ncaa.com/game/lacrosse-women/d1/2019/04/13/army-navy/boxscore
2                https://www.ncaa.com/game/lacrosse-women/d1/2019/05/02/army-navy/boxscore
3      https://www.ncaa.com/game/lacrosse-women/d1/2019/03/06/boston-college-navy/boxscore
4            https://www.ncaa.com/game/lacrosse-women/d1/2019/04/25/navy-boston-u/boxscore
5               https://www.ncaa.com/game/lacrosse-women/d1/2019/03/09/brown-navy/boxscore
6            https://www.ncaa.com/game/lacrosse-women/d1/2019/03/23/bucknell-navy/boxscore
7             https://www.ncaa.com/game/lacrosse-women/d1/2019/03/16/navy-colgate/boxscore
8             https://www.ncaa.com/game/lacrosse-women/d1/2019/03/02/florida-navy/boxscore
9          https://www.ncaa.com/game/lacrosse-women/d1/2019/04/10/navy-georgetown/boxscore
10         https://www.ncaa.com/game/lacrosse-women/d1/2019/05/10/high-point-navy/boxscore